In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pingouin as pg


This just imports the pre and post surveys as pandas dataframes.

In [2]:
path = "/Users/erikmenke/UC-Merced/Data/2018-2019/NSF HSI/2018-2019StudentData/2019Spring/Survey/"
PreSurveyDataFilename = "Presurvey.xlsx"
PostSurveyDataFilename = "Postsurvey.xlsx"

PreSurveyData = pd.read_excel(path+PreSurveyDataFilename)
PostSurveyData = pd.read_excel(path+PostSurveyDataFilename)


First, I'm going to create boolean masks based on the factor analysis to create different groups for the dataframes. The first mask will pick out questions 1, 4, and 5, which all share a common factor. The second mask will pick out questions 7, 11, and 14.

In [3]:
#First mask picks out questions 1, 4, and 5
factor2Mask1 = [True] + 2*[False] + 2*[True] + 15*[False]

#Second mask picks out questions 7, 11, and 14
factor2Mask2 = 6*[False] + [True] + 3*[False] + [True] + 2*[False] + [True] + 6*[False]



Get the loadings for the attitudes on chemistry for the presemester survey

In [4]:
ChemAttPre = PreSurveyData.iloc[:,1:21][1:]
ChemAttPre.dropna(inplace=True)
ChemAttPre = ChemAttPre.astype(str).astype(int)

#This is the first set of factors, reversing question 1
ChemPreFactor1Frame = ChemAttPre.iloc[:,factor2Mask1]
ChemPreFactor1Frame.loc[:,'Q1_1'] = ChemPreFactor1Frame.loc[:,'Q1_1'].add(-8).mul(-1)

#This is the second set of factors, reversing questions 7, 11, and 14
ChemPreFactor2Frame = ChemAttPre.iloc[:,factor2Mask2]
ChemPreFactor2Frame.loc[:,'Q1_7'] = ChemPreFactor2Frame.loc[:,'Q1_7'].add(-8).mul(-1)
ChemPreFactor2Frame.loc[:,'Q1_11'] = ChemPreFactor2Frame.loc[:,'Q1_11'].add(-8).mul(-1)
ChemPreFactor2Frame.loc[:,'Q1_14'] = ChemPreFactor2Frame.loc[:,'Q1_14'].add(-8).mul(-1)

print(pg.cronbach_alpha(ChemPreFactor1Frame))
print(pg.cronbach_alpha(ChemPreFactor2Frame))

print(ChemPreFactor1Frame.corr())
print(ChemPreFactor2Frame.corr())


(0.7741276954516825, array([0.743, 0.802]))
(0.791923341251934, array([0.764, 0.817]))
          Q1_1      Q1_4      Q1_5
Q1_1  1.000000  0.451107  0.559212
Q1_4  0.451107  1.000000  0.601789
Q1_5  0.559212  0.601789  1.000000
           Q1_7     Q1_11     Q1_14
Q1_7   1.000000  0.614945  0.521277
Q1_11  0.614945  1.000000  0.570127
Q1_14  0.521277  0.570127  1.000000


/anaconda3/envs/stats/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Here's the code for the post semester survey, for the chemistry attitudes portion

In [5]:
ChemAttPost = PostSurveyData.iloc[:,1:21][1:]
ChemAttPost.dropna(inplace=True)
ChemAttPost = ChemAttPost.astype(str).astype(int)

#This is the first set of factors, reversing question 1
ChemPostFactor1Frame = ChemAttPost.iloc[:,factor2Mask1]
ChemPostFactor1Frame.loc[:,'Q1_1'] = ChemPostFactor1Frame.loc[:,'Q1_1'].add(-8).mul(-1)

#This is the second set of factors, reversing questions 7, 11, and 14
ChemPostFactor2Frame = ChemAttPost.iloc[:,factor2Mask2]
ChemPostFactor2Frame.loc[:,'Q1_7'] = ChemPostFactor2Frame.loc[:,'Q1_7'].add(-8).mul(-1)
ChemPostFactor2Frame.loc[:,'Q1_11'] = ChemPostFactor2Frame.loc[:,'Q1_11'].add(-8).mul(-1)
ChemPostFactor2Frame.loc[:,'Q1_14'] = ChemPostFactor2Frame.loc[:,'Q1_14'].add(-8).mul(-1)

print(pg.cronbach_alpha(ChemPostFactor1Frame))
print(pg.cronbach_alpha(ChemPostFactor2Frame))

print(ChemPostFactor1Frame.corr())
print(ChemPostFactor2Frame.corr())


(0.8147079978974742, array([0.764, 0.856]))
(0.8373340603963746, array([0.793, 0.874]))
          Q1_1      Q1_4      Q1_5
Q1_1  1.000000  0.509355  0.624809
Q1_4  0.509355  1.000000  0.677974
Q1_5  0.624809  0.677974  1.000000
           Q1_7     Q1_11     Q1_14
Q1_7   1.000000  0.707692  0.536726
Q1_11  0.707692  1.000000  0.690710
Q1_14  0.536726  0.690710  1.000000


/anaconda3/envs/stats/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [6]:
#Calculate the pre and post means for factor 1, as well as effect size
ChemPreFactor1Mean = (ChemPreFactor1Frame).mean().mean()
ChemPreFactor1DOF = (ChemPreFactor1Frame.count()-1).sum()
ChemPreFactor1Variance = ((ChemPreFactor1Frame.std() ** 2) * (ChemPreFactor1Frame.count() - 1)).sum() / ChemPreFactor1DOF

ChemPostFactor1Mean = (ChemPostFactor1Frame).mean().mean()
ChemPostFactor1DOF = (ChemPostFactor1Frame.count()-1).sum()
ChemPostFactor1Variance = ((ChemPostFactor1Frame.std() ** 2) * (ChemPostFactor1Frame.count() - 1)).sum() / ChemPostFactor1DOF

ChemFactor1PooledSD = np.sqrt((ChemPostFactor1DOF * ChemPreFactor1Variance + ChemPostFactor1DOF * ChemPostFactor1Variance) / (ChemPostFactor1DOF + ChemPostFactor1DOF))

ChemFactor1EffectSize = (ChemPostFactor1Mean - ChemPreFactor1Mean)/ChemFactor1PooledSD

#Calculate the pre and post means for factor 2, as well as effect size
ChemPreFactor2Mean = (ChemPreFactor2Frame).mean().mean()
ChemPreFactor2DOF = (ChemPreFactor2Frame.count()-1).sum()
ChemPreFactor2Variance = ((ChemPreFactor2Frame.std() ** 2) * (ChemPreFactor2Frame.count() - 1)).sum() / ChemPreFactor2DOF

ChemPostFactor2Mean = (ChemPostFactor2Frame).mean().mean()
ChemPostFactor2DOF = (ChemPostFactor2Frame.count()-1).sum()
ChemPostFactor2Variance = ((ChemPostFactor2Frame.std() ** 2) * (ChemPostFactor2Frame.count() - 1)).sum() / ChemPostFactor2DOF

ChemFactor2PooledSD = np.sqrt((ChemPostFactor2DOF * ChemPreFactor2Variance + ChemPostFactor2DOF * ChemPostFactor2Variance) / (ChemPostFactor2DOF + ChemPostFactor2DOF))

ChemFactor2EffectSize = (ChemPostFactor2Mean - ChemPreFactor2Mean)/ChemFactor2PooledSD




Now, trying this with the bio attitudes section of the presurvey

In [7]:
BioAttPre = PreSurveyData.iloc[:,21:41][1:]
BioAttPre.dropna(inplace=True)
BioAttPre = BioAttPre.astype(str).astype(int)

#This is the first set of factors, reversing question 1
BioPreFactor1Frame = BioAttPre.iloc[:,factor2Mask1]
BioPreFactor1Frame.loc[:,'Q34_1'] = BioPreFactor1Frame.loc[:,'Q34_1'].add(-8).mul(-1)

#This is the second set of factors, reversing questions 7, 11, and 14
BioPreFactor2Frame = BioAttPre.iloc[:,factor2Mask2]
BioPreFactor2Frame.loc[:,'Q34_7'] = BioPreFactor2Frame.loc[:,'Q34_7'].add(-8).mul(-1)
BioPreFactor2Frame.loc[:,'Q34_11'] = BioPreFactor2Frame.loc[:,'Q34_11'].add(-8).mul(-1)
BioPreFactor2Frame.loc[:,'Q34_14'] = BioPreFactor2Frame.loc[:,'Q34_14'].add(-8).mul(-1)

print(pg.cronbach_alpha(BioPreFactor1Frame))
print(pg.cronbach_alpha(BioPreFactor2Frame))

print(BioPreFactor1Frame.corr())
print(BioPreFactor2Frame.corr())


(0.766304330865634, array([0.734, 0.795]))
(0.7921877054921842, array([0.764, 0.818]))
          Q34_1     Q34_4     Q34_5
Q34_1  1.000000  0.479603  0.530608
Q34_4  0.479603  1.000000  0.580463
Q34_5  0.530608  0.580463  1.000000
           Q34_7    Q34_11    Q34_14
Q34_7   1.000000  0.616967  0.531550
Q34_11  0.616967  1.000000  0.538796
Q34_14  0.531550  0.538796  1.000000


/anaconda3/envs/stats/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Now for the postsemester survey, for the Bio attitudes

In [8]:
BioAttPost = PostSurveyData.iloc[:,21:41][1:]
BioAttPost.dropna(inplace=True)
BioAttPost = BioAttPost.astype(str).astype(int)

#This is the first set of factors, reversing question 1
BioPostFactor1Frame = BioAttPost.iloc[:,factor2Mask1]
BioPostFactor1Frame.loc[:,'Q34_1'] = BioPostFactor1Frame.loc[:,'Q34_1'].add(-8).mul(-1)

#This is the second set of factors, reversing questions 7, 11, and 4
BioPostFactor2Frame = BioAttPost.iloc[:,factor2Mask2]
BioPostFactor2Frame.loc[:,'Q34_7'] = BioPostFactor2Frame.loc[:,'Q34_7'].add(-8).mul(-1)
BioPostFactor2Frame.loc[:,'Q34_11'] = BioPostFactor2Frame.loc[:,'Q34_11'].add(-8).mul(-1)
BioPostFactor2Frame.loc[:,'Q34_14'] = BioPostFactor2Frame.loc[:,'Q34_14'].add(-8).mul(-1)

print(pg.cronbach_alpha(BioPostFactor1Frame))
print(pg.cronbach_alpha(BioPostFactor2Frame))

print(BioPostFactor1Frame.corr())
print(BioPostFactor2Frame.corr())


(0.7769327695906111, array([0.713, 0.828]))
(0.877791563275435, array([0.843, 0.906]))
          Q34_1     Q34_4     Q34_5
Q34_1  1.000000  0.478917  0.540404
Q34_4  0.478917  1.000000  0.611555
Q34_5  0.540404  0.611555  1.000000
           Q34_7    Q34_11    Q34_14
Q34_7   1.000000  0.749058  0.668840
Q34_11  0.749058  1.000000  0.739075
Q34_14  0.668840  0.739075  1.000000


/anaconda3/envs/stats/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [9]:
#Calculate the pre and post means for factor 1, as well as effect size
BioPreFactor1Mean = (BioPreFactor1Frame).mean().mean()
BioPreFactor1DOF = (BioPreFactor1Frame.count()-1).sum()
BioPreFactor1Variance = ((BioPreFactor1Frame.std() ** 2) * (BioPreFactor1Frame.count() - 1)).sum() / BioPreFactor1DOF

BioPostFactor1Mean = (BioPostFactor1Frame).mean().mean()
BioPostFactor1DOF = (BioPostFactor1Frame.count()-1).sum()
BioPostFactor1Variance = ((BioPostFactor1Frame.std() ** 2) * (BioPostFactor1Frame.count() - 1)).sum() / BioPostFactor1DOF

BioFactor1PooledSD = np.sqrt((BioPostFactor1DOF * BioPreFactor1Variance + BioPostFactor1DOF * BioPostFactor1Variance) / (BioPostFactor1DOF + BioPostFactor1DOF))

BioFactor1EffectSize = (BioPostFactor1Mean - BioPreFactor1Mean)/BioFactor1PooledSD

#Calculate the pre and post means for factor 2, as well as effect size
BioPreFactor2Mean = (BioPreFactor2Frame).mean().mean()
BioPreFactor2DOF = (BioPreFactor2Frame.count()-1).sum()
BioPreFactor2Variance = ((BioPreFactor2Frame.std() ** 2) * (BioPreFactor2Frame.count() - 1)).sum() / BioPreFactor2DOF

BioPostFactor2Mean = (BioPostFactor2Frame).mean().mean()
BioPostFactor2DOF = (BioPostFactor2Frame.count()-1).sum()
BioPostFactor2Variance = ((BioPostFactor2Frame.std() ** 2) * (BioPostFactor2Frame.count() - 1)).sum() / BioPostFactor2DOF

BioFactor2PooledSD = np.sqrt((BioPostFactor2DOF * BioPreFactor2Variance + BioPostFactor2DOF * BioPostFactor2Variance) / (BioPostFactor2DOF + BioPostFactor2DOF))

BioFactor2EffectSize = (BioPostFactor2Mean - BioPreFactor2Mean)/BioFactor2PooledSD



Finally, the math attitudes section. This is the presurvey portion

In [10]:
MathAttPre = PreSurveyData.iloc[:,41:61][1:]
MathAttPre.dropna(inplace=True)
MathAttPre = MathAttPre.astype(str).astype(int)

#This is the first set of factors, reversing question 1
MathPreFactor1Frame = MathAttPre.iloc[:,factor2Mask1]
MathPreFactor1Frame.loc[:,'Q32_1'] = MathPreFactor1Frame.loc[:,'Q32_1'].add(-8).mul(-1)

#This is the second set of factors, reversing questions 7, 11, and 14
MathPreFactor2Frame = MathAttPre.iloc[:,factor2Mask2]
MathPreFactor2Frame.loc[:,'Q32_7'] = MathPreFactor2Frame.loc[:,'Q32_7'].add(-8).mul(-1)
MathPreFactor2Frame.loc[:,'Q32_11'] = MathPreFactor2Frame.loc[:,'Q32_11'].add(-8).mul(-1)
MathPreFactor2Frame.loc[:,'Q32_14'] = MathPreFactor2Frame.loc[:,'Q32_14'].add(-8).mul(-1)

print(pg.cronbach_alpha(MathPreFactor1Frame))
print(pg.cronbach_alpha(MathPreFactor2Frame))

print(MathPreFactor1Frame.corr())
print(MathPreFactor2Frame.corr())


/anaconda3/envs/stats/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


(0.8230830676215375, array([0.799, 0.845]))
(0.8461522383900308, array([0.825, 0.865]))
          Q32_1     Q32_4     Q32_5
Q32_1  1.000000  0.598340  0.587169
Q32_4  0.598340  1.000000  0.662338
Q32_5  0.587169  0.662338  1.000000
           Q32_7    Q32_11    Q32_14
Q32_7   1.000000  0.645108  0.621118
Q32_11  0.645108  1.000000  0.708807
Q32_14  0.621118  0.708807  1.000000


Now for the postsemester survey, for the math attitudes

In [11]:
MathAttPost = PostSurveyData.iloc[:,41:61][1:]
MathAttPost.dropna(inplace=True)
MathAttPost = MathAttPost.astype(str).astype(int)

#This is the first set of factors, reversing question 1
MathPostFactor1Frame = MathAttPost.iloc[:,factor2Mask1]
MathPostFactor1Frame.loc[:,'Q32_1'] = MathPostFactor1Frame.loc[:,'Q32_1'].add(-8).mul(-1)

#This is the second set of factors, reversing questions 7, 11, and 14
MathPostFactor2Frame = MathAttPost.iloc[:,factor2Mask2]
MathPostFactor2Frame.loc[:,'Q32_7'] = MathPostFactor2Frame.loc[:,'Q32_7'].add(-8).mul(-1)
MathPostFactor2Frame.loc[:,'Q32_11'] = MathPostFactor2Frame.loc[:,'Q32_11'].add(-8).mul(-1)
MathPostFactor2Frame.loc[:,'Q32_14'] = MathPostFactor2Frame.loc[:,'Q32_14'].add(-8).mul(-1)

print(pg.cronbach_alpha(MathPostFactor1Frame))
print(pg.cronbach_alpha(MathPostFactor2Frame))

print(MathPostFactor1Frame.corr())
print(MathPostFactor2Frame.corr())


/anaconda3/envs/stats/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


(0.8747768346111717, array([0.839, 0.904]))
(0.874127683017032, array([0.838, 0.903]))
          Q32_1     Q32_4     Q32_5
Q32_1  1.000000  0.700676  0.684215
Q32_4  0.700676  1.000000  0.727900
Q32_5  0.684215  0.727900  1.000000
           Q32_7    Q32_11    Q32_14
Q32_7   1.000000  0.750106  0.677991
Q32_11  0.750106  1.000000  0.688000
Q32_14  0.677991  0.688000  1.000000


In [12]:
#Calculate the pre and post means for factor 1, as well as effect size
MathPreFactor1Mean = (MathPreFactor1Frame).mean().mean()
MathPreFactor1DOF = (MathPreFactor1Frame.count()-1).sum()
MathPreFactor1Variance = ((MathPreFactor1Frame.std() ** 2) * (MathPreFactor1Frame.count() - 1)).sum() / MathPreFactor1DOF

MathPostFactor1Mean = (MathPostFactor1Frame).mean().mean()
MathPostFactor1DOF = (MathPostFactor1Frame.count()-1).sum()
MathPostFactor1Variance = ((MathPostFactor1Frame.std() ** 2) * (MathPostFactor1Frame.count() - 1)).sum() / MathPostFactor1DOF

MathFactor1PooledSD = np.sqrt((MathPostFactor1DOF * MathPreFactor1Variance + MathPostFactor1DOF * MathPostFactor1Variance) / (MathPostFactor1DOF + MathPostFactor1DOF))

MathFactor1EffectSize = (MathPostFactor1Mean - MathPreFactor1Mean)/MathFactor1PooledSD

#Calculate the pre and post means for factor 2, as well as effect size
MathPreFactor2Mean = (MathPreFactor2Frame).mean().mean()
MathPreFactor2DOF = (MathPreFactor2Frame.count()-1).sum()
MathPreFactor2Variance = ((MathPreFactor2Frame.std() ** 2) * (MathPreFactor2Frame.count() - 1)).sum() / MathPreFactor2DOF

MathPostFactor2Mean = (MathPostFactor2Frame).mean().mean()
MathPostFactor2DOF = (MathPostFactor2Frame.count()-1).sum()
MathPostFactor2Variance = ((MathPostFactor2Frame.std() ** 2) * (MathPostFactor2Frame.count() - 1)).sum() / MathPostFactor2DOF

MathFactor2PooledSD = np.sqrt((MathPostFactor2DOF * MathPreFactor2Variance + MathPostFactor2DOF * MathPostFactor2Variance) / (MathPostFactor2DOF + MathPostFactor2DOF))

MathFactor2EffectSize = (MathPostFactor2Mean - MathPreFactor2Mean)/MathFactor2PooledSD



In [13]:
#The means and effect sizes for each factor in the three surveys
AttitudeCompare = pd.DataFrame(columns = ['Subscale','Presemester', 'Postsemester', 'Effect size'])

#This is a spacer
AttitudeCompare = AttitudeCompare.append( [{'Subscale': 'Factors', \
                             'Presemester': '', \
                             'Postsemester': '', \
                             'Effect size': ''}]\
                          )

#This is the first factor for chemistry
AttitudeCompare = AttitudeCompare.append( [{'Subscale': 'Cognition, CHEM', \
                             'Presemester': 100*ChemPreFactor1Mean/7, \
                             'Postsemester': 100*ChemPostFactor1Mean/7, \
                             'Effect size': ChemFactor1EffectSize}]\
                          )

#This is the first factor for bio
AttitudeCompare = AttitudeCompare.append( [{'Subscale': 'Cognition, BIO', \
                             'Presemester': 100*BioPreFactor1Mean/7, \
                             'Postsemester': 100*BioPostFactor1Mean/7, \
                             'Effect size': BioFactor1EffectSize}]\
                          )

#This is the first factor for math
AttitudeCompare = AttitudeCompare.append( [{'Subscale': 'Cognition, MATH', \
                             'Presemester': 100*MathPreFactor1Mean/7, \
                             'Postsemester': 100*MathPostFactor1Mean/7, \
                             'Effect size': MathFactor1EffectSize}]\
                          )

#This is the second factor for chemistry
AttitudeCompare = AttitudeCompare.append( [{'Subscale': 'Affect, CHEM', \
                             'Presemester': 100*ChemPreFactor2Mean/7, \
                             'Postsemester': 100*ChemPostFactor2Mean/7, \
                             'Effect size': ChemFactor2EffectSize}]\
                          )

#This is the second factor for bio
AttitudeCompare = AttitudeCompare.append( [{'Subscale': 'Affect, BIO', \
                             'Presemester': 100*BioPreFactor2Mean/7, \
                             'Postsemester': 100*BioPostFactor2Mean/7, \
                             'Effect size': BioFactor2EffectSize}]\
                          )

#This is the second factor for math
AttitudeCompare = AttitudeCompare.append( [{'Subscale': 'Affect, MATH', \
                             'Presemester': 100*MathPreFactor2Mean/7, \
                             'Postsemester': 100*MathPostFactor2Mean/7, \
                             'Effect size': MathFactor2EffectSize}]\
                          )




In [22]:
AttitudeCompare

#ExportFilename = "AttitudeCompare_Menke.xlsx"
#AttitudeCompare.to_excel(path + ExportFilename)



In [15]:
#The means and standard deviations for each factor in the three surveys
AttitudeData = pd.DataFrame(columns = ['Subscale','Presemester mean', \
                                       'Presemester SD', \
                                       'Postsemester mean', \
                                       'Postsemester SD'])

#This is a spacer
AttitudeData = AttitudeData.append( [{'Subscale': 'Factors', \
                             'Presemester mean': '', \
                             'Presemester SD': '',\
                             'Postsemester mean': '',\
                             'Postsemester SD': ''}]\
                          )

#This is the first factor for chemistry
AttitudeData = AttitudeData.append( [{'Subscale': 'Cognition, CHEM', \
                             'Presemester mean': ChemPreFactor1Mean, \
                             'Presemester SD': np.sqrt(ChemPreFactor1Variance), \
                             'Postsemester mean': ChemPostFactor1Mean, \
                             'Postsemester SD': np.sqrt(ChemPostFactor1Variance)}]\
                          )

#This is the first factor for bio
AttitudeData = AttitudeData.append( [{'Subscale': 'Cognition, BIO', \
                             'Presemester mean': BioPreFactor1Mean, \
                             'Presemester SD': np.sqrt(BioPreFactor1Variance), \
                             'Postsemester mean': BioPostFactor1Mean, \
                             'Postsemester SD': np.sqrt(BioPostFactor1Variance)}]\
                          )

#This is the first factor for math
AttitudeData = AttitudeData.append( [{'Subscale': 'Cognition, MATH', \
                             'Presemester mean': MathPreFactor1Mean, \
                             'Presemester SD': np.sqrt(MathPreFactor1Variance), \
                             'Postsemester mean': MathPostFactor1Mean, \
                             'Postsemester SD': np.sqrt(MathPostFactor1Variance)}]\
                          )

#This is the second factor for chemistry
AttitudeData = AttitudeData.append( [{'Subscale': 'Affect, CHEM', \
                             'Presemester mean': ChemPreFactor2Mean, \
                             'Presemester SD': np.sqrt(ChemPreFactor2Variance), \
                             'Postsemester mean': ChemPostFactor2Mean, \
                             'Postsemester SD': np.sqrt(ChemPostFactor2Variance)}]\
                          )

#This is the second factor for bio
AttitudeData = AttitudeData.append( [{'Subscale': 'Affect, BIO', \
                             'Presemester mean': BioPreFactor2Mean, \
                             'Presemester SD': np.sqrt(BioPreFactor2Variance), \
                             'Postsemester mean': BioPostFactor2Mean, \
                             'Postsemester SD': np.sqrt(BioPostFactor2Variance)}]\
                          )

#This is the second factor for math
AttitudeData = AttitudeData.append( [{'Subscale': 'Affect, MATH', \
                             'Presemester mean': MathPreFactor2Mean, \
                             'Presemester SD': np.sqrt(MathPreFactor2Variance), \
                             'Postsemester mean': MathPostFactor2Mean, \
                             'Postsemester SD': np.sqrt(MathPostFactor2Variance)}]\
                          )



In [21]:
AttitudeData

#ExportFilename = "AttitudeData_Menke.xlsx"
#AttitudeData.to_excel(path + ExportFilename)



In [17]:
#Calculate the presemester t-value between Chem and Bio for factor 1
ChemBioPrePooledVar1 = (ChemPreFactor1DOF * ChemPreFactor1Variance + BioPreFactor1DOF * BioPreFactor1Variance) / (ChemPreFactor1DOF + BioPreFactor1DOF)
ChemBioPreN1 = (1/ChemPreFactor1DOF) + (1/BioPreFactor1DOF)
ChemBioPreT1 = (ChemPreFactor1Mean - BioPreFactor1Mean)/np.sqrt(ChemBioPrePooledVar1*ChemBioPreN1)

#Calculate the presemester t-value between Chem and Math for factor 1
ChemMathPrePooledVar1 = (ChemPreFactor1DOF * ChemPreFactor1Variance + MathPreFactor1DOF * MathPreFactor1Variance) / (ChemPreFactor1DOF + MathPreFactor1DOF)
ChemMathPreN1 = (1/ChemPreFactor1DOF) + (1/MathPreFactor1DOF)
ChemMathPreT1 = (ChemPreFactor1Mean - MathPreFactor1Mean)/np.sqrt(ChemMathPrePooledVar1*ChemMathPreN1)

#Calculate the presemester t-value between Bio and Math for factor 1
BioMathPrePooledVar1 = (BioPreFactor1DOF * BioPreFactor1Variance + MathPreFactor1DOF * MathPreFactor1Variance) / (BioPreFactor1DOF + MathPreFactor1DOF)
BioMathPreN1 = (1/BioPreFactor1DOF) + (1/BioPreFactor1DOF)
BioMathPreT1 = (BioPreFactor1Mean - MathPreFactor1Mean)/np.sqrt(BioMathPrePooledVar1*BioMathPreN1)

#Calculate the presemester t-value between Chem and Bio for factor 2
ChemBioPrePooledVar2 = (ChemPreFactor2DOF * ChemPreFactor2Variance + BioPreFactor2DOF * BioPreFactor2Variance) / (ChemPreFactor2DOF + BioPreFactor2DOF)
ChemBioPreN2 = (1/ChemPreFactor2DOF) + (1/BioPreFactor2DOF)
ChemBioPreT2 = (ChemPreFactor2Mean - BioPreFactor2Mean)/np.sqrt(ChemBioPrePooledVar2*ChemBioPreN2)

#Calculate the presemester t-value between Chem and Math for factor 2
ChemMathPrePooledVar2 = (ChemPreFactor2DOF * ChemPreFactor2Variance + MathPreFactor2DOF * MathPreFactor2Variance) / (ChemPreFactor2DOF + MathPreFactor2DOF)
ChemMathPreN2 = (1/ChemPreFactor2DOF) + (1/MathPreFactor2DOF)
ChemMathPreT2 = (ChemPreFactor2Mean - MathPreFactor2Mean)/np.sqrt(ChemMathPrePooledVar2*ChemMathPreN2)

#Calculate the presemester t-value between Bio and Math for factor 2
BioMathPrePooledVar2 = (BioPreFactor2DOF * BioPreFactor2Variance + MathPreFactor2DOF * MathPreFactor2Variance) / (BioPreFactor2DOF + MathPreFactor2DOF)
BioMathPreN2 = (1/BioPreFactor2DOF) + (1/BioPreFactor2DOF)
BioMathPreT2 = (BioPreFactor2Mean - MathPreFactor2Mean)/np.sqrt(BioMathPrePooledVar2*BioMathPreN2)

#Calculate the postsemester t-value between Chem and Bio for factor 1
ChemBioPostPooledVar1 = (ChemPostFactor1DOF * ChemPostFactor1Variance + BioPostFactor1DOF * BioPostFactor1Variance) / (ChemPostFactor1DOF + BioPostFactor1DOF)
ChemBioPostN1 = (1/ChemPostFactor1DOF) + (1/BioPostFactor1DOF)
ChemBioPostT1 = (ChemPostFactor1Mean - BioPostFactor1Mean)/np.sqrt(ChemBioPostPooledVar1*ChemBioPostN1)

#Calculate the postsemester t-value between Chem and Math for factor 1
ChemMathPostPooledVar1 = (ChemPostFactor1DOF * ChemPostFactor1Variance + MathPostFactor1DOF * MathPostFactor1Variance) / (ChemPostFactor1DOF + MathPostFactor1DOF)
ChemMathPostN1 = (1/ChemPostFactor1DOF) + (1/MathPostFactor1DOF)
ChemMathPostT1 = (ChemPostFactor1Mean - MathPostFactor1Mean)/np.sqrt(ChemMathPostPooledVar1*ChemMathPostN1)

#Calculate the postsemester t-value between Bio and Math for factor 1
BioMathPostPooledVar1 = (BioPostFactor1DOF * BioPostFactor1Variance + MathPostFactor1DOF * MathPostFactor1Variance) / (BioPostFactor1DOF + MathPostFactor1DOF)
BioMathPostN1 = (1/BioPostFactor1DOF) + (1/BioPostFactor1DOF)
BioMathPostT1 = (BioPostFactor1Mean - MathPostFactor1Mean)/np.sqrt(BioMathPostPooledVar1*BioMathPostN1)

#Calculate the postsemester t-value between Chem and Bio for factor 2
ChemBioPostPooledVar2 = (ChemPostFactor2DOF * ChemPostFactor2Variance + BioPostFactor2DOF * BioPostFactor2Variance) / (ChemPostFactor2DOF + BioPostFactor2DOF)
ChemBioPostN2 = (1/ChemPostFactor2DOF) + (1/BioPostFactor2DOF)
ChemBioPostT2 = (ChemPostFactor2Mean - BioPostFactor2Mean)/np.sqrt(ChemBioPostPooledVar2*ChemBioPostN2)

#Calculate the postsemester t-value between Chem and Math for factor 2
ChemMathPostPooledVar2 = (ChemPostFactor2DOF * ChemPostFactor2Variance + MathPostFactor2DOF * MathPostFactor2Variance) / (ChemPostFactor2DOF + MathPostFactor2DOF)
ChemMathPostN2 = (1/ChemPostFactor2DOF) + (1/MathPostFactor2DOF)
ChemMathPostT2 = (ChemPostFactor2Mean - MathPostFactor2Mean)/np.sqrt(ChemMathPostPooledVar2*ChemMathPostN2)

#Calculate the postsemester t-value between Bio and Math for factor 2
BioMathPostPooledVar2 = (BioPostFactor2DOF * BioPostFactor2Variance + MathPostFactor2DOF * MathPostFactor2Variance) / (BioPostFactor2DOF + MathPostFactor2DOF)
BioMathPostN2 = (1/BioPostFactor2DOF) + (1/BioPostFactor2DOF)
BioMathPostT2 = (BioPostFactor2Mean - MathPostFactor2Mean)/np.sqrt(BioMathPostPooledVar2*BioMathPostN2)



In [18]:
#The means and standard deviations for each factor in the three surveys
TCompare = pd.DataFrame(columns = ['Subscale','Presemester t-value', \
                              'Presemester DOF', \
                                      'Postsemester t-value', \
                                       'Postsemester DOF'])

#This is a spacer
TCompare = TCompare.append( [{'Subscale': 'Factors', \
                             'Presemester t-value': '', \
                             'Presemester DOF': '', \
                             'Postsemester t-value': '',\
                             'Postsemester DOF': ''}]\
                          )

#This is the first factor for chemistry
TCompare = TCompare.append( [{'Subscale': 'Cognition, CHEM-BIO', \
                             'Presemester t-value': ChemBioPreT1, \
                             'Presemester DOF': ChemPreFactor1DOF + BioPreFactor1DOF, \
                             'Postsemester t-value': ChemBioPostT1, \
                             'Postsemester DOF': ChemPostFactor1DOF + BioPostFactor1DOF}]\
                          )

#This is the first factor for bio
TCompare = TCompare.append( [{'Subscale': 'Cognition, CHEM-MATH', \
                             'Presemester t-value': ChemMathPreT1, \
                             'Presemester DOF': ChemPreFactor1DOF + MathPreFactor1DOF, \
                             'Postsemester t-value': ChemMathPostT1, \
                             'Postsemester DOF': ChemPostFactor1DOF + MathPostFactor1DOF}]\
                          )

#This is the first factor for math
TCompare = TCompare.append( [{'Subscale': 'Cognition, BIO-MATH', \
                             'Presemester t-value': BioMathPreT1, \
                             'Presemester DOF': BioPreFactor1DOF + MathPreFactor1DOF, \
                             'Postsemester t-value': BioMathPostT1, \
                             'Postsemester DOF': BioPostFactor1DOF + MathPostFactor1DOF}]\
                          )

#This is the second factor for chemistry
TCompare = TCompare.append( [{'Subscale': 'Affect, CHEM-BIO', \
                             'Presemester t-value': ChemBioPreT2, \
                             'Presemester DOF': ChemPreFactor2DOF + BioPreFactor2DOF, \
                             'Postsemester t-value': ChemBioPostT2, \
                             'Postsemester DOF': ChemPostFactor2DOF + BioPostFactor2DOF}]\
                          )

#This is the second factor for bio
TCompare = TCompare.append( [{'Subscale': 'Affect, CHEM-MATH', \
                             'Presemester t-value': ChemMathPreT2, \
                             'Presemester DOF': ChemPreFactor2DOF + MathPreFactor2DOF, \
                             'Postsemester t-value': ChemMathPostT2, \
                             'Postsemester DOF': ChemPostFactor2DOF + MathPostFactor2DOF}]\
                          )

#This is the second factor for math
TCompare = TCompare.append( [{'Subscale': 'Affect, BIO-MATH', \
                             'Presemester t-value': BioMathPreT2, \
                             'Presemester DOF': BioPreFactor2DOF + MathPreFactor2DOF, \
                             'Postsemester t-value': BioMathPostT2, \
                             'Postsemester DOF': BioPostFactor2DOF + MathPostFactor2DOF}]\
                          )



In [24]:
TCompare

#ExportFilename = "TCompare_Menke.xlsx"
#TCompare.to_excel(path + ExportFilename)



,Subscale,Presemester t-value,Presemester DOF,Postsemester t-value,Postsemester DOF
0,Factors,,,,
0,"Cognition, CHEM-BIO",-6.29745,4161,-2.69709,1098
0,"Cognition, CHEM-MATH",3.97441,4146,0.315055,1086
0,"Cognition, BIO-MATH",9.21479,4119,2.57385,1050
0,"Affect, CHEM-BIO",-9.35576,4161,-3.50464,1098
0,"Affect, CHEM-MATH",1.56254,4146,-1.26147,1086
0,"Affect, BIO-MATH",9.8713,4119,1.82785,1050
